In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout

2023-06-26 21:26:39.762144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 21:26:39.976008: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 21:26:39.977260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 21:26:41.727482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data Preparation

In [2]:
indices_train = np.load('../data/indices_train.npy')

X_full = np.stack(list(map(lambda index: np.load(f'../data/images_train/image_{index:03d}.npy'), range(40))))
y = np.stack(list(map(lambda index: np.load(f'../data/masks_train/mask_{index:03d}.npy'), range(40))))
# The labels do not change
y_train = y[indices_train[:, 0], 0, indices_train[:, 2], indices_train[:, 3]]

## Create 5x5 Patches

In [3]:
KERNEL_RADIUS = 2
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_5x5 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_5x5 = np.reshape(X_train_5x5, ((X_train_5x5.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_5x5)
std = np.std(X_train_5x5)
# Normalize the data
X_train_5x5 = (X_train_5x5 - mean) / std

print(X_train_5x5.shape)

(34976, 5, 5, 10)


## Create 7x7 Patches

In [4]:
KERNEL_RADIUS = 3
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_7x7 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_7x7 = np.reshape(X_train_7x7, ((X_train_7x7.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_7x7)
std = np.std(X_train_7x7)
# Normalize the data
X_train_7x7 = (X_train_7x7 - mean) / std

print(X_train_7x7.shape)

(34976, 7, 7, 10)


## Create 9x9 Patches

In [5]:
KERNEL_RADIUS = 4
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_9x9 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_9x9 = np.reshape(X_train_9x9, ((X_train_9x9.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_9x9)
std = np.std(X_train_9x9)
# Normalize the data
X_train_9x9 = (X_train_9x9 - mean) / std

print(X_train_9x9.shape)

(34976, 9, 9, 10)


# Model Definition

The models get bigger and bigger with larger indices.

In [7]:
def get_run_logdir(model_id):
    return os.path.join(os.curdir, f'cnn_logs/model_{model_id}')

earlystopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./cnn_logs --port=6006

In [8]:
model = {}

## Model 1: 5x5 Input, 3x3x3 Kernel, 10 Dense

In [19]:
i = 1
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 3, 3, 3)           273       
                                                                 
 flatten_8 (Flatten)         (None, 27)                0         
                                                                 
 dense_21 (Dense)            (None, 10)                280       
                                                                 
 dense_22 (Dense)            (None, 1)                 11        
                                                                 
Total params: 564
Trainable params: 564
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_1/assets


INFO:tensorflow:Assets written to: cnn_models/model_1/assets


## Model 2: 5x5 Input, 3x3x8 Kernel, 10 Dense

In [20]:
i = 2
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 conv2d_9 (Conv2D)           (None, 3, 3, 8)           728       
                                                                 
 flatten_9 (Flatten)         (None, 72)                0         
                                                                 
 dense_23 (Dense)            (None, 10)                730       
                                                                 
 dense_24 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,469
Trainable params: 1,469
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_2/assets


INFO:tensorflow:Assets written to: cnn_models/model_2/assets


## Model 3: 5x5 Input, 3x3x12 Kernel, 10 Dense

In [21]:
i = 3
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 3, 3, 12)          1092      
                                                                 
 flatten_10 (Flatten)        (None, 108)               0         
                                                                 
 dense_25 (Dense)            (None, 10)                1090      
                                                                 
 dense_26 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2,193
Trainable params: 2,193
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_3/assets


INFO:tensorflow:Assets written to: cnn_models/model_3/assets


## Model 4: 5x5 Input, 3x3x8 Kernel, 50 Dense, 10 Dense

In [22]:
i = 4
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 3, 3, 8)           728       
                                                                 
 flatten_11 (Flatten)        (None, 72)                0         
                                                                 
 dense_27 (Dense)            (None, 50)                3650      
                                                                 
 dense_28 (Dense)            (None, 10)                510       
                                                                 
 dense_29 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4,899
Trainable params: 4,899
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_4/assets


INFO:tensorflow:Assets written to: cnn_models/model_4/assets


## Model 5: 5x5 Input, 3x3x8 Kernel, 0.2 Dropout, 50 Dense, 10 Dense

In [23]:
i = 5
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 3, 3, 8)           728       
                                                                 
 flatten_12 (Flatten)        (None, 72)                0         
                                                                 
 dropout_3 (Dropout)         (None, 72)                0         
                                                                 
 dense_30 (Dense)            (None, 50)                3650      
                                                                 
 dense_31 (Dense)            (None, 10)                510       
                                                                 
 dense_32 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4,899
Trainable params: 4,899
Non-train

2023-06-20 17:30:53.098228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,72]
	 [[{{node inputs}}]]
2023-06-20 17:30:53.231527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,72]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_5/assets


INFO:tensorflow:Assets written to: cnn_models/model_5/assets


## Model 6: 5x5 Input, 3x3x8 Kernel, 0.1 Dropout, 50 Dense, 10 Dense

In [24]:
i = 6
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 3, 3, 8)           728       
                                                                 
 flatten_13 (Flatten)        (None, 72)                0         
                                                                 
 dropout_4 (Dropout)         (None, 72)                0         
                                                                 
 dense_33 (Dense)            (None, 50)                3650      
                                                                 
 dense_34 (Dense)            (None, 10)                510       
                                                                 
 dense_35 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4,899
Trainable params: 4,899
Non-train

2023-06-20 17:33:21.697621: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,72]
	 [[{{node inputs}}]]
2023-06-20 17:33:21.843350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,72]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_6/assets


INFO:tensorflow:Assets written to: cnn_models/model_6/assets


## Model 7: 7x7 Input, 3x3x3 Kernel, 10 Dense

In [25]:
i = 7
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 5, 5, 3)           273       
                                                                 
 flatten_14 (Flatten)        (None, 75)                0         
                                                                 
 dense_36 (Dense)            (None, 10)                760       
                                                                 
 dense_37 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________


2023-06-20 17:35:44.302708: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54840800 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: cnn_models/model_7/assets


INFO:tensorflow:Assets written to: cnn_models/model_7/assets


## Model 8: 7x7 Input, 3x3x8 Kernel, 10 Dense

In [26]:
i = 8
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 5, 5, 8)           728       
                                                                 
 flatten_15 (Flatten)        (None, 200)               0         
                                                                 
 dense_38 (Dense)            (None, 10)                2010      
                                                                 
 dense_39 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2,749
Trainable params: 2,749
Non-trainable params: 0
_________________________________________________________________


2023-06-20 17:37:49.910016: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54840800 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: cnn_models/model_8/assets


INFO:tensorflow:Assets written to: cnn_models/model_8/assets


## Model 9: 7x7 Input, 3x3x8 Kernel, 50 Dense, 10 Dense

In [27]:
i = 9
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 5, 5, 8)           728       
                                                                 
 flatten_16 (Flatten)        (None, 200)               0         
                                                                 
 dense_40 (Dense)            (None, 50)                10050     
                                                                 
 dense_41 (Dense)            (None, 10)                510       
                                                                 
 dense_42 (Dense)            (None, 1)                 11        
                                                                 
Total params: 11,299
Trainable params: 11,299
Non-trainable params: 0
_________________________________________________________________


2023-06-20 17:40:47.002086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54840800 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: cnn_models/model_9/assets


INFO:tensorflow:Assets written to: cnn_models/model_9/assets


## Model 10: 7x7 Input, 3x3x8 Kernel, 0.2 Dropout, 50 Dense, 10 Dense

In [28]:
i = 10
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 5, 5, 8)           728       
                                                                 
 flatten_17 (Flatten)        (None, 200)               0         
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_43 (Dense)            (None, 50)                10050     
                                                                 
 dense_44 (Dense)            (None, 10)                510       
                                                                 
 dense_45 (Dense)            (None, 1)                 11        
                                                                 
Total params: 11,299
Trainable params: 11,299
Non-tra

2023-06-20 17:42:46.654759: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54840800 exceeds 10% of free system memory.
2023-06-20 17:45:21.062725: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200]
	 [[{{node inputs}}]]
2023-06-20 17:45:21.187366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_10/assets


INFO:tensorflow:Assets written to: cnn_models/model_10/assets


## Model 11: 7x7 Input, 3x3x8 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [29]:
i = 11
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 5, 5, 8)           728       
                                                                 
 flatten_18 (Flatten)        (None, 200)               0         
                                                                 
 dropout_6 (Dropout)         (None, 200)               0         
                                                                 
 dense_46 (Dense)            (None, 50)                10050     
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_47 (Dense)            (None, 10)                510       
                                                                 
 dense_48 (Dense)            (None, 1)               

2023-06-20 17:45:21.681548: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54840800 exceeds 10% of free system memory.
2023-06-20 17:46:12.419504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200]
	 [[{{node inputs}}]]
2023-06-20 17:46:12.431436: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-20 17:46:12.559290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID

INFO:tensorflow:Assets written to: cnn_models/model_11/assets


INFO:tensorflow:Assets written to: cnn_models/model_11/assets


## Model 12: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 10 Dense

In [30]:
i = 12
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_19 (Flatten)        (None, 300)               0         
                                                                 
 dropout_8 (Dropout)         (None, 300)               0         
                                                                 
 dense_49 (Dense)            (None, 100)               30100     
                                                                 
 dropout_9 (Dropout)         (None, 100)               0         
                                                                 
 dense_50 (Dense)            (None, 50)                5050      
                                                                 
 dense_51 (Dense)            (None, 10)              

2023-06-20 17:51:36.372782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 17:51:36.395679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-20 17:51:36.831962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 17:51

INFO:tensorflow:Assets written to: cnn_models/model_12/assets


INFO:tensorflow:Assets written to: cnn_models/model_12/assets


## Model 13: 7x7 Input, 3x3x12 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 10 Dense

In [32]:
i = 13
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_21 (Flatten)        (None, 300)               0         
                                                                 
 dropout_12 (Dropout)        (None, 300)               0         
                                                                 
 dense_57 (Dense)            (None, 100)               30100     
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_58 (Dense)            (None, 50)                5050      
                                                                 
 dense_59 (Dense)            (None, 10)              

2023-06-20 17:56:26.509243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 17:56:26.534389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-20 17:56:26.839212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 17:56

INFO:tensorflow:Assets written to: cnn_models/model_13/assets


INFO:tensorflow:Assets written to: cnn_models/model_13/assets


## Model 14: 7x7 Input, 3x3x12 Kernel, 20 Dense

In [33]:
i = 14
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_22 (Flatten)        (None, 300)               0         
                                                                 
 dense_61 (Dense)            (None, 20)                6020      
                                                                 
 dense_62 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7,133
Trainable params: 7,133
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_14/assets


INFO:tensorflow:Assets written to: cnn_models/model_14/assets


## Model 15: 7x7 Input, 3x3x12 Kernel, 0.05 Dropout, 20 Dense

In [34]:
i = 15
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_23 (Flatten)        (None, 300)               0         
                                                                 
 dropout_14 (Dropout)        (None, 300)               0         
                                                                 
 dense_63 (Dense)            (None, 20)                6020      
                                                                 
 dense_64 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7,133
Trainable params: 7,133
Non-trainable params: 0
_________________________________________________________________


2023-06-20 18:00:43.350795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 18:00:43.550044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_15/assets


INFO:tensorflow:Assets written to: cnn_models/model_15/assets


## Model 16: 7x7 Input, 3x3x12 Kernel, 0.05 Dropout, 40 Dense

In [35]:
i = 16
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(40, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_24 (Flatten)        (None, 300)               0         
                                                                 
 dropout_15 (Dropout)        (None, 300)               0         
                                                                 
 dense_65 (Dense)            (None, 40)                12040     
                                                                 
 dense_66 (Dense)            (None, 1)                 41        
                                                                 
Total params: 13,173
Trainable params: 13,173
Non-trainable params: 0
_________________________________________________________________


2023-06-20 18:02:29.307222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 18:02:29.498636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_16/assets


INFO:tensorflow:Assets written to: cnn_models/model_16/assets


## Model 17: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 80 Dense

In [36]:
i = 17
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(80, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_25 (Flatten)        (None, 300)               0         
                                                                 
 dropout_16 (Dropout)        (None, 300)               0         
                                                                 
 dense_67 (Dense)            (None, 80)                24080     
                                                                 
 dense_68 (Dense)            (None, 1)                 81        
                                                                 
Total params: 25,253
Trainable params: 25,253
Non-trainable params: 0
_________________________________________________________________


2023-06-20 18:06:21.909556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 18:06:22.100356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_17/assets


INFO:tensorflow:Assets written to: cnn_models/model_17/assets


## Model 18: 7x7 Input, 3x3x12 Kernel, 0.15 Dropout, 120 Dense

In [37]:
i = 18
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.15),
    Dense(120, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_26 (Flatten)        (None, 300)               0         
                                                                 
 dropout_17 (Dropout)        (None, 300)               0         
                                                                 
 dense_69 (Dense)            (None, 120)               36120     
                                                                 
 dense_70 (Dense)            (None, 1)                 121       
                                                                 
Total params: 37,333
Trainable params: 37,333
Non-trainable params: 0
_________________________________________________________________


2023-06-20 18:09:05.567160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-20 18:09:05.707053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_18/assets


INFO:tensorflow:Assets written to: cnn_models/model_18/assets


## Model 19: 9x9 Input, 3x3x3 Kernel, 10 Dense

In [38]:
i = 19
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 7, 7, 3)           273       
                                                                 
 flatten_27 (Flatten)        (None, 147)               0         
                                                                 
 dense_71 (Dense)            (None, 10)                1480      
                                                                 
 dense_72 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,764
Trainable params: 1,764
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_19/assets


INFO:tensorflow:Assets written to: cnn_models/model_19/assets


## Model 20: 9x9 Input, 3x3x8 Kernel, 10 Dense

In [39]:
i = 20
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 7, 7, 8)           728       
                                                                 
 flatten_28 (Flatten)        (None, 392)               0         
                                                                 
 dense_73 (Dense)            (None, 10)                3930      
                                                                 
 dense_74 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4,669
Trainable params: 4,669
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_20/assets


INFO:tensorflow:Assets written to: cnn_models/model_20/assets


## Model 21: 9x9 Input, 5x5x8 Kernel, 10 Dense

In [40]:
i = 21
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=5, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 5, 5, 8)           2008      
                                                                 
 flatten_29 (Flatten)        (None, 200)               0         
                                                                 
 dense_75 (Dense)            (None, 10)                2010      
                                                                 
 dense_76 (Dense)            (None, 1)                 11        
                                                                 
Total params: 4,029
Trainable params: 4,029
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_21/assets


INFO:tensorflow:Assets written to: cnn_models/model_21/assets


## Model 22: 9x9 Input, 5x5x8 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 10 Dense

In [41]:
i = 22
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=5, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 5, 5, 8)           2008      
                                                                 
 flatten_30 (Flatten)        (None, 200)               0         
                                                                 
 dropout_18 (Dropout)        (None, 200)               0         
                                                                 
 dense_77 (Dense)            (None, 100)               20100     
                                                                 
 dropout_19 (Dropout)        (None, 100)               0         
                                                                 
 dense_78 (Dense)            (None, 10)                1010      
                                                                 
 dense_79 (Dense)            (None, 1)               

2023-06-20 18:16:01.299157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200]
	 [[{{node inputs}}]]
2023-06-20 18:16:01.321310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-20 18:16:01.595087: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200]
	 [[{{node inputs}}]]
2023-06-20 18:16

INFO:tensorflow:Assets written to: cnn_models/model_22/assets


INFO:tensorflow:Assets written to: cnn_models/model_22/assets


## Model 23: 9x9 Input, 3x3x8 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [9]:
i = 23
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 8)           728       
                                                                 
 flatten (Flatten)           (None, 392)               0         
                                                                 
 dropout (Dropout)           (None, 392)               0         
                                                                 
 dense (Dense)               (None, 100)               39300     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_2 (Dropout)         (None, 50)                0

2023-06-26 21:27:09.274889: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90655200 exceeds 10% of free system memory.
2023-06-26 21:31:56.961035: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,392]
	 [[{{node inputs}}]]
2023-06-26 21:31:57.000157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-26 21:31:57.039503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALI

INFO:tensorflow:Assets written to: cnn_models/model_23/assets


INFO:tensorflow:Assets written to: cnn_models/model_23/assets


## Model 24: 9x9 Input, 3x3x8 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 0.2 Dropout, 10 Dense

In [10]:
i = 24
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 flatten_1 (Flatten)         (None, 392)               0         
                                                                 
 dropout_3 (Dropout)         (None, 392)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               39300     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dropout_5 (Dropout)         (None, 50)               

2023-06-26 21:33:10.220941: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90655200 exceeds 10% of free system memory.
2023-06-26 21:39:40.211568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,392]
	 [[{{node inputs}}]]
2023-06-26 21:39:40.259496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-26 21:39:40.303665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALI

INFO:tensorflow:Assets written to: cnn_models/model_24/assets


INFO:tensorflow:Assets written to: cnn_models/model_24/assets


## Model 25: 9x9 Input, 3x3x8 Kernel, 0.3 Dropout, 100 Dense, 0.3 Dropout, 50 Dense, 0.3 Dropout, 10 Dense

In [11]:
i = 25
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 flatten_2 (Flatten)         (None, 392)               0         
                                                                 
 dropout_6 (Dropout)         (None, 392)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               39300     
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dropout_8 (Dropout)         (None, 50)               

2023-06-26 21:40:20.409050: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90655200 exceeds 10% of free system memory.


KeyboardInterrupt: 

## Model 26: 9x9 Input, 3x3x8 Kernel, 0.1 Dropout, 30 Dense, 0.1 Dropout, 10 Dense

In [12]:
i = 26
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(30, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 flatten_3 (Flatten)         (None, 392)               0         
                                                                 
 dropout_9 (Dropout)         (None, 392)               0         
                                                                 
 dense_12 (Dense)            (None, 30)                11790     
                                                                 
 dropout_10 (Dropout)        (None, 30)                0         
                                                                 
 dense_13 (Dense)            (None, 10)                310       
                                                                 
 dense_14 (Dense)            (None, 1)                

2023-06-26 21:44:34.143931: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90655200 exceeds 10% of free system memory.
2023-06-26 21:48:04.967272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,392]
	 [[{{node inputs}}]]
2023-06-26 21:48:05.009584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,30]
	 [[{{node inputs}}]]
2023-06-26 21:48:05.513909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID

INFO:tensorflow:Assets written to: cnn_models/model_26/assets


INFO:tensorflow:Assets written to: cnn_models/model_26/assets


## Model 27: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 10 Dense

In [13]:
i = 27
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 conv2d_5 (Conv2D)           (None, 5, 5, 2)           146       
                                                                 
 flatten_4 (Flatten)         (None, 50)                0         
                                                                 
 dense_15 (Dense)            (None, 10)                510       
                                                                 
 dense_16 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,395
Trainable params: 1,395
Non-trainable params: 0
_________________________________________________________________


2023-06-26 21:52:19.270176: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90655200 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: cnn_models/model_27/assets


INFO:tensorflow:Assets written to: cnn_models/model_27/assets


## Model 28: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 20 Dense, 10 Dense

In [14]:
i = 28
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 conv2d_7 (Conv2D)           (None, 5, 5, 2)           146       
                                                                 
 flatten_5 (Flatten)         (None, 50)                0         
                                                                 
 dense_17 (Dense)            (None, 20)                1020      
                                                                 
 dense_18 (Dense)            (None, 10)                210       
                                                                 
 dense_19 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2,115
Trainable params: 2,115
Non-traina

INFO:tensorflow:Assets written to: cnn_models/model_28/assets


INFO:tensorflow:Assets written to: cnn_models/model_28/assets


## Model 29: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [15]:
i = 29
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 7, 7, 8)           728       
                                                                 
 conv2d_9 (Conv2D)           (None, 5, 5, 2)           146       
                                                                 
 flatten_6 (Flatten)         (None, 50)                0         
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 dense_20 (Dense)            (None, 50)                2550      
                                                                 
 dropout_12 (Dropout)        (None, 50)                0         
                                                                 
 dense_21 (Dense)            (None, 10)               

2023-06-26 22:04:32.426500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:04:32.436127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:04:32.584690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:04:32

INFO:tensorflow:Assets written to: cnn_models/model_29/assets


INFO:tensorflow:Assets written to: cnn_models/model_29/assets


## Model 30: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [16]:
i = 30
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 7, 7, 8)           728       
                                                                 
 conv2d_11 (Conv2D)          (None, 5, 5, 2)           146       
                                                                 
 flatten_7 (Flatten)         (None, 50)                0         
                                                                 
 dropout_13 (Dropout)        (None, 50)                0         
                                                                 
 dense_23 (Dense)            (None, 100)               5100      
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_24 (Dense)            (None, 50)               

2023-06-26 22:15:28.111182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:15:28.152441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-26 22:15:28.193733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:15:2

INFO:tensorflow:Assets written to: cnn_models/model_30/assets


INFO:tensorflow:Assets written to: cnn_models/model_30/assets


## Model 31: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [17]:
i = 31
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 5, 5, 12)          1092      
                                                                 
 flatten_8 (Flatten)         (None, 300)               0         
                                                                 
 dropout_16 (Dropout)        (None, 300)               0         
                                                                 
 dense_27 (Dense)            (None, 50)                15050     
                                                                 
 dropout_17 (Dropout)        (None, 50)                0         
                                                                 
 dense_28 (Dense)            (None, 10)                510       
                                                                 
 dense_29 (Dense)            (None, 1)                

2023-06-26 22:24:50.160938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-26 22:24:50.171285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:24:50.298074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,300]
	 [[{{node inputs}}]]
2023-06-26 22:24:

INFO:tensorflow:Assets written to: cnn_models/model_31/assets


INFO:tensorflow:Assets written to: cnn_models/model_31/assets


## Model 32: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [18]:
i = 32
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_9 (Flatten)         (None, 500)               0         
                                                                 
 dropout_18 (Dropout)        (None, 500)               0         
                                                                 
 dense_30 (Dense)            (None, 50)                25050     
                                                                 
 dropout_19 (Dropout)        (None, 50)                0         
                                                                 
 dense_31 (Dense)            (None, 10)                510       
                                                                 
 dense_32 (Dense)            (None, 1)                

2023-06-26 22:28:49.678566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-26 22:28:49.690315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:28:49.814404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-26 22:28:

INFO:tensorflow:Assets written to: cnn_models/model_32/assets


INFO:tensorflow:Assets written to: cnn_models/model_32/assets


## Model 33: 7x7 Input, 3x3x20 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 0.2 Dropout, 10 Dense

In [19]:
i = 33
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_10 (Flatten)        (None, 500)               0         
                                                                 
 dropout_20 (Dropout)        (None, 500)               0         
                                                                 
 dense_33 (Dense)            (None, 100)               50100     
                                                                 
 dropout_21 (Dropout)        (None, 100)               0         
                                                                 
 dense_34 (Dense)            (None, 50)                5050      
                                                                 
 dropout_22 (Dropout)        (None, 50)              

2023-06-26 22:32:04.429808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-26 22:32:04.441930: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-26 22:32:04.453880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:32:

INFO:tensorflow:Assets written to: cnn_models/model_33/assets


INFO:tensorflow:Assets written to: cnn_models/model_33/assets


## Model 34: 7x7 Input, 3x3x20 Kernel, 0.3 Dropout, 100 Dense, 0.3 Dropout, 50 Dense, 0.3 Dropout, 10 Dense

In [20]:
i = 34
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_11 (Flatten)        (None, 500)               0         
                                                                 
 dropout_23 (Dropout)        (None, 500)               0         
                                                                 
 dense_37 (Dense)            (None, 100)               50100     
                                                                 
 dropout_24 (Dropout)        (None, 100)               0         
                                                                 
 dense_38 (Dense)            (None, 50)                5050      
                                                                 
 dropout_25 (Dropout)        (None, 50)              

2023-06-26 22:40:38.341175: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-26 22:40:38.351941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,100]
	 [[{{node inputs}}]]
2023-06-26 22:40:38.361613: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,50]
	 [[{{node inputs}}]]
2023-06-26 22:40:

INFO:tensorflow:Assets written to: cnn_models/model_34/assets


INFO:tensorflow:Assets written to: cnn_models/model_34/assets


## Model 35: 7x7 Input, 3x3x20 Kernel, 0.25 Dropout, 100 Dense, 10 Dense

In [21]:
i = 35
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.25),
    Dense(100, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_12 (Flatten)        (None, 500)               0         
                                                                 
 dropout_26 (Dropout)        (None, 500)               0         
                                                                 
 dense_41 (Dense)            (None, 100)               50100     
                                                                 
 dense_42 (Dense)            (None, 10)                1010      
                                                                 
 dense_43 (Dense)            (None, 1)                 11        
                                                                 
Total params: 52,941
Trainable params: 52,941
Non-tra

2023-06-27 09:08:11.834758: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 09:08:12.434970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_35/assets


INFO:tensorflow:Assets written to: cnn_models/model_35/assets


## Model 36: 7x7 Input, 3x3x20 Kernel, 0.2 Dropout, 75 Dense, 25 Dense

In [22]:
i = 36
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(75, activation='relu'),
    Dense(25, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_13 (Flatten)        (None, 500)               0         
                                                                 
 dropout_27 (Dropout)        (None, 500)               0         
                                                                 
 dense_44 (Dense)            (None, 75)                37575     
                                                                 
 dense_45 (Dense)            (None, 25)                1900      
                                                                 
 dense_46 (Dense)            (None, 1)                 26        
                                                                 
Total params: 41,321
Trainable params: 41,321
Non-tra

KeyboardInterrupt: 

## Model 37: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 20 Dense, 10 Dense

In [23]:
i = 37
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(20, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_14 (Flatten)        (None, 500)               0         
                                                                 
 dropout_28 (Dropout)        (None, 500)               0         
                                                                 
 dense_47 (Dense)            (None, 20)                10020     
                                                                 
 dense_48 (Dense)            (None, 10)                210       
                                                                 
 dense_49 (Dense)            (None, 1)                 11        
                                                                 
Total params: 12,061
Trainable params: 12,061
Non-tra

2023-06-27 09:17:51.401235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 09:17:51.880098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_37/assets


INFO:tensorflow:Assets written to: cnn_models/model_37/assets


## Model 38: 7x7 Input, 3x3x20 Kernel, 10 Dense

In [24]:
i = 38
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_15 (Flatten)        (None, 500)               0         
                                                                 
 dense_50 (Dense)            (None, 10)                5010      
                                                                 
 dense_51 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,841
Trainable params: 6,841
Non-trainable params: 0
_________________________________________________________________


INFO:tensorflow:Assets written to: cnn_models/model_38/assets


INFO:tensorflow:Assets written to: cnn_models/model_38/assets


## Model 39: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 10 Dense

In [25]:
i = 39
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_16 (Flatten)        (None, 500)               0         
                                                                 
 dropout_29 (Dropout)        (None, 500)               0         
                                                                 
 dense_52 (Dense)            (None, 10)                5010      
                                                                 
 dense_53 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,841
Trainable params: 6,841
Non-trainable params: 0
_________________________________________________________________


2023-06-27 09:23:17.864747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 09:23:18.006770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_39/assets


INFO:tensorflow:Assets written to: cnn_models/model_39/assets


## Model 40: 7x7 Input, 3x3x20 Kernel, 0.15 Dropout, 20 Dense

In [26]:
i = 40
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.15),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_17 (Flatten)        (None, 500)               0         
                                                                 
 dropout_30 (Dropout)        (None, 500)               0         
                                                                 
 dense_54 (Dense)            (None, 20)                10020     
                                                                 
 dense_55 (Dense)            (None, 1)                 21        
                                                                 
Total params: 11,861
Trainable params: 11,861
Non-trainable params: 0
_________________________________________________________________


2023-06-27 09:28:28.859844: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 09:28:29.300534: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_40/assets


INFO:tensorflow:Assets written to: cnn_models/model_40/assets


## Model 41: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 25 Dense

In [27]:
i = 41
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(25, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_18 (Flatten)        (None, 500)               0         
                                                                 
 dropout_31 (Dropout)        (None, 500)               0         
                                                                 
 dense_56 (Dense)            (None, 25)                12525     
                                                                 
 dense_57 (Dense)            (None, 1)                 26        
                                                                 
Total params: 14,371
Trainable params: 14,371
Non-trainable params: 0
_________________________________________________________________


2023-06-27 09:31:26.934705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 09:31:27.135236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_models/model_41/assets


INFO:tensorflow:Assets written to: cnn_models/model_41/assets


# Best Model: 39

Very good fit, comparably few parameters, fast training.

In [31]:
model = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model.summary()
model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model.fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(0)), earlystopping_cb])
model.save('../models/cnn')

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 5, 5, 20)          1820      
                                                                 
 flatten_22 (Flatten)        (None, 500)               0         
                                                                 
 dropout_35 (Dropout)        (None, 500)               0         
                                                                 
 dense_64 (Dense)            (None, 10)                5010      
                                                                 
 dense_65 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,841
Trainable params: 6,841
Non-trainable params: 0
_________________________________________________________________


2023-06-27 10:05:14.906255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-27 10:05:15.062410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: ../models/cnn/assets


INFO:tensorflow:Assets written to: ../models/cnn/assets
